In [1]:
import pandas as pd
import datetime
from tqdm import tqdm

In [ ]:
{
    "name": "4c6bc1bc7651",
    "cluster_name": "docker-cluster",
    "cluster_uuid": "artY2Nl2T-SW_5rtBCcAJw",
    "version": {
        "number": "7.9.2",
        "build_flavor": "default",
        "build_type": "docker",
        "build_hash": "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",
        "build_date": "2020-09-23T00:45:33.626720Z",
        "build_snapshot": false,
        "lucene_version": "8.6.2",
        "minimum_wire_compatibility_version": "6.8.0",
        "minimum_index_compatibility_version": "6.0.0-beta1"
    },
    "tagline": "You Know, for Search"
}

In [2]:
import json

x = {
    'title': 'asbasd',
    'genres': ['d', 'f']
}

json.dumps(x)

'{"title": "asbasd", "genres": ["d", "f"]}'

In [2]:
df = pd.read_csv('movies.csv')
df['title'] = df['title'].map(lambda x: x.strip())
df['genres'] = df['genres'].map(lambda x: x.split('|'))
# df['year'] = df['title'].map(lambda x: x[-6:][1:-1])
df = df.astype({'title': 'str'})
df.iloc[0:1].to_json()

'{"movieId":{"0":1},"title":{"0":"Toy Story (1995)"},"genres":{"0":["Adventure","Animation","Children","Comedy","Fantasy"]}}'

In [17]:
datetime.datetime.fromtimestamp(1260759144)

datetime.datetime(2009, 12, 14, 4, 52, 24)

In [3]:
df = pd.read_csv('ratings.csv')
df['datetime'] = df['timestamp'].map(lambda x: datetime.datetime.fromtimestamp(x))
df.dtypes, df.head()

(userId                int64
 movieId               int64
 rating              float64
 timestamp             int64
 datetime     datetime64[ns]
 dtype: object,
    userId  movieId  rating   timestamp            datetime
 0       1       31     2.5  1260759144 2009-12-14 04:52:24
 1       1     1029     3.0  1260759179 2009-12-14 04:52:59
 2       1     1061     3.0  1260759182 2009-12-14 04:53:02
 3       1     1129     2.0  1260759185 2009-12-14 04:53:05
 4       1     1172     4.0  1260759205 2009-12-14 04:53:25)

In [54]:
df.iloc[0:1].to_json()

'{"movieId":{"1":1},"title":{"1":"Toy Story (1995)"},"genres":{"1":["Adventure","Animation","Children","Comedy","Fantasy"]}}'

In [4]:
import datetime
import json
import requests

import pandas as pd
import numpy as np 

from elasticsearch import Elasticsearch
res = requests.get('http://localhost:9200')
print(res.content)

es = Elasticsearch([{'host': 'localhost', 'port': 9200}])

b'{\n  "name" : "ed85cc06ed5c",\n  "cluster_name" : "docker-cluster",\n  "cluster_uuid" : "5vApkKCLTmmQYiio85zHww",\n  "version" : {\n    "number" : "7.9.2",\n    "build_flavor" : "default",\n    "build_type" : "docker",\n    "build_hash" : "d34da0ea4a966c4e49417f2da2f244e3e97b4e6e",\n    "build_date" : "2020-09-23T00:45:33.626720Z",\n    "build_snapshot" : false,\n    "lucene_version" : "8.6.2",\n    "minimum_wire_compatibility_version" : "6.8.0",\n    "minimum_index_compatibility_version" : "6.0.0-beta1"\n  },\n  "tagline" : "You Know, for Search"\n}\n'


In [5]:
es.indices.create(index='movies', ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'movies'}

In [6]:
mapping = {
            'properties': {
                'title': {'type': 'text'}
            }
        }
es.indices.put_mapping(body=json.dumps(mapping), index='movies', doc_type='movie', include_type_name=True)

{'acknowledged': True}

In [7]:
es.indices.create(index='ratings',ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'ratings'}

In [8]:
mapping = {
    'properties': {
        'datetime': {
            'type': 'date',
            'format': 'yyyy-MM-dd HH:mm:ss'
        },
    },
    # '_parent': {
    #     'type': 'movie'
    # }
}
es.indices.put_mapping(json.dumps(mapping), index='ratings', doc_type='rating', include_type_name=True)

{'acknowledged': True}

In [12]:
df = pd.read_csv('movies.csv')
df['title'] = df['title'].map(lambda x: x.strip())
df['genres'] = df['genres'].map(lambda x: x.split('|'))
# df['year'] = df['title'].map(lambda x: x[-6:][1:-1])
df = df.astype({'title': 'str'})

df_movies = df

df = pd.read_csv('ratings.csv')
df['datetime'] = df['timestamp'].map(lambda x: datetime.datetime.fromtimestamp(x))

df_ratings = df

In [10]:
for idx, row in tqdm(df_movies.iterrows()):
    title = row['title']
    movieId = row['movieId']
    genres = row['genres']

    entry = {
        'title': title,
        'genres': genres
    }

    es.index(index='movies', doc_type='movie', id=movieId, body=json.dumps(entry), request_timeout=60)

9125it [01:30, 100.50it/s]


In [11]:
for idx, row in tqdm(df_ratings.iterrows()):
    tt = row['datetime']
    rating = row['rating']
    userId = row['userId']
    movieId = row['movieId']

    entry = {
        'datetime': str(tt),
        'rating': rating,
        'userId': userId,
        'movieId': movieId
    }

    es.index(index='ratings', doc_type='rating', id=f'{userId}-{movieId}', body=json.dumps(entry))

100004it [13:11, 126.38it/s]


In [19]:
q = 'dark'
query = {
    'query': {
        'match': {
            'title': q
        }
    },
    'sort': [{'_score': 'desc'}]
}
res = es.search(index='movies', doc_type='movie', body=json.dumps(query), params={'size': 20})
res

{'took': 18,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 33, 'relation': 'eq'},
  'max_score': 6.3944416,
  'hits': [{'_index': 'movies',
    '_type': 'movie',
    '_id': '1748',
    '_score': 6.3944416,
    '_source': {'title': 'Dark City (1998)',
     'genres': ['Adventure', 'Film-Noir', 'Sci-Fi', 'Thriller']}},
   {'_index': 'movies',
    '_type': 'movie',
    '_id': '3727',
    '_score': 6.3944416,
    '_source': {'title': 'Near Dark (1987)', 'genres': ['Horror', 'Western']}},
   {'_index': 'movies',
    '_type': 'movie',
    '_id': '6185',
    '_score': 6.3944416,
    '_source': {'title': 'Dark Blue (2003)',
     'genres': ['Action', 'Crime', 'Drama', 'Thriller']}},
   {'_index': 'movies',
    '_type': 'movie',
    '_id': '7061',
    '_score': 6.3944416,
    '_source': {'title': 'Dark Victory (1939)',
     'genres': ['Drama', 'Romance']}},
   {'_index': 'movies',
    '_type': 'movie',
    '_id': '7217',
  

In [30]:
userIds = df_ratings['userId'].unique()
movieIds = df_ratings['movieId'].unique()

ratings = df_ratings['rating'].unique()

genres = set()

for idx, row in df_movies.iterrows():
    genres.update(row['genres'])
genres

{'(no genres listed)',
 'Action',
 'Adventure',
 'Animation',
 'Children',
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'IMAX',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

In [31]:
ratings

array([2.5, 3. , 2. , 4. , 3.5, 1. , 5. , 4.5, 1.5, 0.5])

In [29]:
len(genres), len(genres)**2, len(userIds), len(movieIds)

(20, 400, 671, 9066)

In [14]:
X = df_ratings[['userId', 'movieId']].values
Y = df_ratings['rating'].values

X.shape, Y.shape

((100004, 2), (100004,))

In [15]:
X[:, 0].shape

(100004,)

In [16]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder

user_one_hot = OneHotEncoder()
movie_one_hot = OneHotEncoder()

user_one_hot.fit(X[:, 0].reshape(-1, 1))
movie_one_hot.fit(X[:, 1].reshape(-1, 1))

X_user = user_one_hot.transform(X[:, 0].reshape(-1, 1))
X_movie = movie_one_hot.transform(X[:, 1].reshape(-1, 1))

X_user.shape, X_movie.shape

((100004, 671), (100004, 9066))

In [18]:
import numpy as np
from scipy.sparse import hstack

X_kmeans = hstack([X_user, X_movie])
X_kmeans.shape

(100004, 9737)

In [19]:
k = 30
kmeans = KMeans(n_clusters=k)
kmeans.fit(X_kmeans, Y)

score = kmeans.inertia_
score

173004.40657043768

In [36]:
scores

{20: 98674139345.47446,
 21: 85116667972.84253,
 22: 78561990214.39313,
 23: 73491933222.61375,
 24: 68023425725.99783,
 25: 64030029429.34196,
 26: 56669748841.0162,
 27: 53954309085.234726,
 28: 48737427563.694405,
 29: 46122856596.77394,
 30: 42147083593.44612,
 31: 39833275602.12562}

In [17]:
import pickle

pickle.dump(user_one_hot, open('user_one_hot.pkl', 'wb'))
pickle.dump(movie_one_hot, open('movie_one_hot.pkl', 'wb'))
#pickle.dump(kmeans, open('kmeans.pkl', 'wb'))

In [24]:
user_one_hot = pickle.load(open('user_one_hot.pkl', 'rb'))
movie_one_hot = pickle.load(open('movie_one_hot.pkl', 'rb'))
kmeans = pickle.load(open('kmeans.pkl', 'rb'))

user_one_hot, movie_one_hot, kmeans

(OneHotEncoder(), OneHotEncoder(), KMeans(n_clusters=30))

In [39]:
import operator
min(scores.items(), key=operator.itemgetter(1))[0]

59

In [38]:
import itertools
predict_keys = [[x, y] for x, y in itertools.product(userIds, movieIds)]

In [40]:
predict_ratings = kmeans.predict(predict_keys)

In [41]:
predict_ratings.shape

(6083286,)

In [42]:
es.indices.create(index='kmeans',ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'kmeans'}

In [48]:
type(predict_keys[0][0].item())

int

In [49]:
for tupl, rating in tqdm(zip(predict_keys, predict_ratings)):
    user_id = tupl[0].item()
    movie_id = tupl[0].item()
    r = rating.item()

    entry = {
        'userId': user_id,
        'movieId': movieId,
        'rating': r
    }

    es.index(index='kmeans', id=f'{userId}-{movieId}', body=json.dumps(entry))

3057896it [5:54:01, 143.52it/s]

KeyboardInterrupt: 

In [ ]:
import tensorflow as tf

X = {'user_id': df_ratings['userId'].values, 'movie_id': df_ratings['movieId'].values}
Y = df_ratings['rating'].values

In [ ]:
input_layers = {
    colname: Input(name=colname, shape=(1,), dtype=tf.int32)
    for colname in ['user_id', 'movie_id']
}

user_id_input = input_layers['user_id']
movie_id_input = input_layers['movie_id']

user_emb = tf.keras.layers.Embedding(len(userIds), 64, name='user_emb')(user_id_input)
movie_emb = tf.keras.layers.Embedding(len(movieIds), 64, name='movie_emb')(movie_id_input)

features = tf.keras.layers.Concatenate()([user_emb, movie_emb])

hidden = tf.keras.layers.Dense(64, activation='relu')(features)
model_out = tf.keras.layers.Dense(1)(hidden)
model_out = tf.keras.layers.LeakyReLU()(model_out)
model = tf.keras.Model(input_layers, model_out)

In [ ]:
history = model.fit(X, Y, epochs=5)

In [ ]:
predict_keys = [{'user_id': x, 'movie_id': y} for x, y in itertools.product(userIds, movieIds)]
predict_values = model.predict(predict_keys)

In [ ]:
predict_values.shape, predict_values.dtype

In [ ]:
type(predict_keys[predict.keys()[0]])